In [ ]:
# hide
!nvidia-smi

Mon Jun 14 18:32:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# hide
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastai transformers datasets wandb seqeval
    !pip install -q git+git://github.com/aikindergarten/fasthugs.git@tok-clf

     |████████████████████████████████| 204kB 4.4MB/s 
     |████████████████████████████████| 2.3MB 7.3MB/s 
     |████████████████████████████████| 245kB 21.9MB/s 
     |████████████████████████████████| 1.8MB 23.7MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 901kB 16.3MB/s 
     |████████████████████████████████| 3.3MB 36.9MB/s 
     |████████████████████████████████| 122kB 40.0MB/s 
     |████████████████████████████████| 245kB 37.5MB/s 
     |████████████████████████████████| 133kB 37.8MB/s 
     |████████████████████████████████| 174kB 41.6MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 


In [ ]:
#all_slow

# Token Classification

In [ ]:
#hide
from transformers import AutoModelForTokenClassification, AutoTokenizer
from datasets import load_dataset, concatenate_datasets
from fastai.text.all import *
from fastai.callback.wandb import WandbCallback
from fasthugs.learner import TransLearner
from fasthugs.data import TokenClassificationBlock, PadTokBatchTransform
from fasthugs.data import get_splits, KeyGetter

## Setup

In [ ]:
ds_name = "conll2003"
model_name = "google/electra-small-discriminator"

max_len = 512
bs = 16
val_bs = bs*2

lr = 3e-5

In [ ]:
#hide_output
ds = load_dataset(ds_name)

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6. Subsequent calls will reuse this data.


## Dataloaders

In [ ]:
splits = get_splits(ds)
ds = concatenate_datasets([ds['train'], ds['validation']])

In [ ]:
ds[0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

In [ ]:
task = 'ner' # 'pos', 'chunk'
label_vocab = ds.features[f"{task}_tags"].feature.names
label_vocab

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
#hide_output
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
label_all_tokens = True

The preprocessing is explained in HuggingFace [example notebook](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb).

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
ds = ds.map(tokenize_and_align_labels, batched=True)

In [ ]:
dblock = DataBlock(
    blocks = [TokenClassificationBlock(tokenizer=tokenizer, label_vocab=label_vocab)],
    get_x=KeyGetter(['input_ids', 'attention_mask', 'token_type_ids', 'labels']),
    splitter=RandomSplitter())

In [ ]:
# dblock.summary(ds)

In [ ]:
%%time
dls = dblock.dataloaders(ds, bs=bs, val_bs=val_bs, num_workes=2)

CPU times: user 49.8 s, sys: 1.55 s, total: 51.3 s
Wall time: 57.4 s


In [ ]:
# hide
b = dls.one_batch()
b[0]['input_ids'].shape

torch.Size([16, 48])

In [ ]:
dls.show_batch(max_n=4)

,text,tags
0,u. n. relief officials said they were not aware that the tanks advancing on arbil were manned by iraqi troops as they advanced from kdp - controlled areas and raised kdp flags.,"B-ORG, B-ORG, B-ORG, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, B-LOC, O, O, O, B-MISC, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG, B-ORG, O, O,"
1,13 - thomas enqvist ( sweden ) vs. stephane simian ( france ),"O, O, B-PER, I-PER, I-PER, O, B-LOC, O, O, O, B-PER, I-PER, I-PER, O, B-LOC, O,"
2,san francisco 56 73. 434 14 1 / 2,"B-ORG, I-ORG, O, O, O, O, O, O, O, O, O,"
3,"daniels pharmaceuticals manufactures prescription pharmaceutical products, the largest of which is levoxyl, a synthetic thyroid hormone for treating hypothyroidism.","B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, B-MISC, B-MISC, B-MISC, O, O, O, O, O, O, O, O, O, O, O, O, O, O,"


## Metrics

In [ ]:
from datasets import load_metric
seqeval = load_metric("seqeval")

In [ ]:
#collaple_output
print(seqeval)

Metric(name: "seqeval", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of shape (n_samples,), weights for indi

In [ ]:
from fasthugs.metrics import MetricCallback
from typing import Tuple

class Seqeval(MetricCallback):

    def __init__(self, label_list, scores:Tuple[str]=('accuracy', 'f1', 'precision', 'recall')):
        self.metric = load_metric('seqeval')
        store_attr()
        self._register_value_funcs()

    @staticmethod
    def _get_score(obj, score, **kwargs):
        return obj.res[f"overall_{score}"]

    def preprocess(self, predictions, labels):

        # Remove ignored index (special tokens)
        true_predictions = [
            [self.label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [self.label_list[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        return true_predictions, true_labels

In [ ]:
#hide
import wandb

WANDB_NAME = f'{ds_name}-{model_name}'
GROUP = f'{ds_name}-{model_name}-{task}-{lr:.0e}'
NOTES = f'finetuning {model_name} with Adam lr={lr:.0e}'
CONFIG = {}
TAGS =[model_name, ds_name, 'adam']

In [ ]:
#hide_output
wandb.init(reinit=True, project="fasthugs", entity="fastai_community",
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Training

In [ ]:
#hide_output
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_vocab))
learn = TransLearner(dls, model, loss_func=noop, cbs=Seqeval(label_vocab))

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.weight', 'c

The final layers of the model are initialized with random weights, we can varufy that the performance is as good as random choice:

In [ ]:
learn.validate()

(#5) [2.1915693283081055,0.12161642694685589,0.03735191637630662,0.022816832887375335,0.10290729566648382]

In [ ]:
# cbs = [WandbCallback(log_preds=False, log_model=False)]
cbs = []
learn.fit_one_cycle(2, lr, wd=0.01, cbs=cbs)

epoch,train_loss,valid_loss,accuracy,f1,precision,recall,time
0,0.197947,0.173893,0.952574,0.776549,0.774937,0.778168,02:01
1,0.147690,0.134610,0.962232,0.837942,0.829961,0.846078,02:01
